# ロジット
- pythonでロジットをする
- 参考文献：
    - 「データサイエンス育成講座」
    - 山本「計量経済学」
    - 黒住「計量経済学」
    - 「Rによる計量経済分析」

普通の回帰で最小二乗法が使えるのは、「ある仮定」が成り立つとき、ガウスマルコフの定理によって、最小二条推定量がBLUEだから。
つまり、最も適切な推定量であるから。

BLUEとは、

- 線形性
- 普遍性
- 効率性
- 一致性

が成り立つこと。

上が成り立つためには誤差項について「ある仮定」が必要:

- 均一分散
- 共分散ゼロ
- 説明変数と独立

被説明変数がダミー変数のとき（線形確率モデル）、
被説明変数が0か1の値しかとらないため、誤差項の分散が説明変数の大きさに応じて変わってしまい、不均一分散となる。

したがって、0か1や確率を予測したい場合、最小二乗法は使えない。

そこで、

- 頑健標準誤差の利用
- 加重最小二乗法
- 一般化最小二乗法

などの工夫をして対処する。

線形確率モデルでは、不均一分散になってしまうのを、加重最小二乗法で解決しようとするが、今度は真の重みがわからないという問題が生じるので、一般化最小二乗法で対処する。
別の問題として、予測値が0から1に収まらない場合もあるので、「確率」として解釈できなくなってしまう。

そこで、非線形モデルを使う。
非線形モデルは回帰曲線が直線ではなく、正規分布やロジスティック分布を用いる。
非線形なので最適化には最小二乗法ではなく、最尤法を使う。（最小二乗法は最尤法の特別な例）

というわけで、ここではプロビットモデルやロジットモデルを使うのいいだろう。

では、被説明変数が3値以上ある場合にはどうすればよいか。

順序がある場合には順序ロジットモデル、順序がない場合には多項ロジットモデルを使う。

以上をまとめると、

**「ロジットモデル」と「多項ロジットモデル」をとりあえずやってみる。**


### ライブラリ

In [1]:
import numpy as np
import scipy as sp
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

%precision 3

'%.3f'

### データのダウンロードと読み込み

In [2]:
import requests, zipfile
from io import StringIO
import io

In [3]:
# データを取得
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
res = requests.get(url).content

# 取得したデータをDataFrameオブジェクトとして読み込み
adult = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# データの列にラベルを設定
adult.columns =['age','workclass','fnlwgt','education','education-num','marital-status',
                             'occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week',
                             'native-country','flg-50K']


# データの形式と欠損数を出力
print('データの形式:{}'.format(adult.shape))
print('欠損の数:{}'.format(adult.isnull().sum().sum()))

# データの先頭5行を出力
adult.head()

データの形式:(32561, 15)
欠損の数:0


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,flg-50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
adult.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [5]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  flg-50K         32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## ロジット

### 分析対象を整理

In [6]:
adult.groupby('flg-50K').size()

flg-50K
 <=50K    24720
 >50K      7841
dtype: int64

In [7]:
# 「fin_flg」カラムを追加し、もし「flg-50K」カラムの値が「>50K」だったら1、そうでなければ0をセットする
adult['fin_flg'] = adult['flg-50K'].map(lambda x: 1 if x ==' >50K' else 0)
adult.groupby('fin_flg').size()

fin_flg
0    24720
1     7841
dtype: int64

### sklearnでロジット

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 説明変数と目的変数の設定
X = adult[['age','fnlwgt','education-num','capital-gain','capital-loss']]
y = adult['fin_flg']

# 訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# ロジスティック回帰クラスの初期化と学習
model = LogisticRegression()
model.fit(X_train,y_train)

print('正解率(train):{:.3f}'.format(model.score(X_train, y_train)))
print('正解率(test):{:.3f}'.format(model.score(X_test, y_test)))

正解率(train):0.797
正解率(test):0.798


- 係数取得

In [9]:
model.coef_

array([[-1.185e-02, -4.379e-06, -2.774e-03,  3.274e-04,  7.532e-04]])

- オッズ比取得

In [10]:
np.exp(model.coef_)

array([[0.988, 1.   , 0.997, 1.   , 1.001]])

- 標準化

In [11]:
# 標準化のためのクラスをインポート
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Xとyを設定
X = adult[['age','fnlwgt','education-num','capital-gain','capital-loss']]
y = adult['fin_flg']

# 訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# ロジスティック回帰クラスの初期化と学習
model = LogisticRegression()
model.fit(X_train_std,y_train)

# 正解率の表示
print('正解率(train):{:.3f}'.format(model.score(X_train_std, y_train)))
print('正解率(test):{:.3f}'.format(model.score(X_test_std, y_test)))

正解率(train):0.811
正解率(test):0.810


### statsmodelsでロジット

In [13]:
import statsmodels.api as sm

model2 = sm.Logit(y_train, sm.add_constant(X_train))
result = model2.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.425198
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                fin_flg   No. Observations:                16280
Model:                          Logit   Df Residuals:                    16274
Method:                           MLE   Df Model:                            5
Date:                Tue, 11 May 2021   Pseudo R-squ.:                  0.2260
Time:                        10:55:41   Log-Likelihood:                -6922.2
converged:                       True   LL-Null:                       -8943.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -6.5229      0.137    -47.575      0.000      -6.792      -6.254
age               0.

- 表記はzになっているが、中身はtvalueっぽいのでtvalueだと思って使おう

In [15]:
margeff = result.get_margeff()
print(margeff.summary())

        Logit Marginal Effects       
Dep. Variable:                fin_flg
Method:                          dydx
At:                           overall
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age               0.0053      0.000     25.368      0.000       0.005       0.006
fnlwgt         3.267e-08   2.78e-08      1.176      0.239   -2.18e-08    8.71e-08
education-num     0.0440      0.001     39.015      0.000       0.042       0.046
capital-gain   4.182e-05   1.72e-06     24.335      0.000    3.85e-05    4.52e-05
capital-loss    9.47e-05   5.79e-06     16.350      0.000    8.34e-05       0.000


In [18]:
result.predict(sm.add_constant(X_train))

7872     0.407626
31180    0.426038
25875    0.244286
4345     0.203548
15278    0.522437
           ...   
13123    0.435077
19648    0.056385
9845     0.087297
10799    0.947891
2732     0.193376
Length: 16280, dtype: float64

In [19]:
result.predict(sm.add_constant(X_test))

22278    0.103541
8950     0.236909
7838     0.172961
16505    0.085540
19140    0.389708
           ...   
30177    0.064362
4107     0.016969
16706    0.363475
19415    0.096667
29823    0.476757
Length: 16281, dtype: float64

In [21]:
result.tvalues

const           -47.574608
age              24.081164
fnlwgt            1.176076
education-num    34.741973
capital-gain     23.022550
capital-loss     15.892291
dtype: float64

In [24]:
result.pvalues

const             0.000000e+00
age              3.938193e-128
fnlwgt            2.395644e-01
education-num    1.832296e-264
capital-gain     2.771573e-117
capital-loss      7.165973e-57
dtype: float64

- ロジットで、（デフォルトが）z値を使うのはなぜか？

- p値で有意水準を判断すればいい

## 多項ロジット